In [987]:
import pandas as pd
import numpy as np

In [988]:
df = pd.read_excel('Для 4го отчета (Ноябрь 2018).xlsx')

In [989]:
df.head()

,ID,CATEGORY_ID,CHECK_TYPE_ID,SRC_ID,DT_LETTER,DT_DECISION
0,153846,ЮЛ,1,13,"11.09.18 12:06:00,000000000","11.09.18 13:03:24,000000000"
1,153847,ФЛ,1,2,"11.09.18 12:43:00,000000000","11.09.18 13:03:51,000000000"
2,153848,ФЛ,5,17,"11.09.18 12:21:00,000000000","11.09.18 13:21:26,000000000"
3,151773,ФЛ,5,17,"29.08.18 18:48:00,000000000","30.08.18 10:35:25,000000000"
4,151774,ЮЛ,1,13,"30.08.18 09:40:00,000000000","30.08.18 10:01:49,000000000"


In [990]:
df['DT_DECISION']= pd.to_datetime(df['DT_DECISION'], dayfirst= True)
df['DT_LETTER']= pd.to_datetime(df['DT_LETTER'], dayfirst= True)

df = df.sort_values(by='DT_LETTER')

# Список выходных дней

In [991]:
auto_weekends = pd.read_excel('АВТО выходные.xlsx')

rest_weekends = pd.read_excel('НЕ АВТО выходные.xlsx')

# Разбивка на недели

In [992]:
from datetime import timedelta
from datetime import datetime

def week_range(date):
    """Find the first/last day of the week for the given day.
    Assuming weeks start on Sunday and end on Saturday.

    Returns a tuple of ``(start_date, end_date)``.

    """
    # isocalendar calculates the year, week of the year, and day of the week.
    # dow is Mon = 1, Sat = 6, Sun = 7
    year, week, dow = date.isocalendar()

    # Find the first day of the week.
    if dow == 1:
        # Since we want to start with Sunday, let's test for that condition.
        start_date = date
    else:
        # Otherwise, subtract `dow` number days to get the first day
        start_date = date - timedelta(dow - 1)

    # Now, add 6 for the last day of the week (i.e., count up to Saturday)
    end_date = start_date + timedelta(6)

    #return (start_date.date(), end_date.date())
    string = ''
    
    if start_date.date().day < 10:
        string = string + '0' + str(start_date.date().day)
    else:
        string = string + str(start_date.date().day)
    
    if start_date.date().month < 10:
        string = string + '.' + '0' + str(start_date.date().month)
    else:
        string = string + '.' + str(start_date.date().month)
    
    string = string + '.' + str(start_date.date().year) + '-'
    
    if end_date.date().day < 10:
        string = string + '0' + str(end_date.date().day)
    else:
        string = string + str(end_date.date().day)
    
    if end_date.date().month < 10:
        string = string + '.' + '0' + str(end_date.date().month)
    else:
        string = string + '.' + str(end_date.date().month)    
    string = string + '.' + str(start_date.date().year)
    
    #return ( str(start_date.date().day) + '.' + str(start_date.date().month) + '.' + str(start_date.date().year) + 
    #       '-' + str(end_date.date().day) + '.' + str(end_date.date().month) + '.' + str(end_date.date().year))
    return string    


df['week'] = df['DT_LETTER'].apply(week_range)

# Список проверок и источников

In [993]:
check_types = {
1: 'АВТО',
2: 'Перевозчик грузов',
3: 'Парк ТС',
4: 'Имущество',
5: 'Кандидат',
6: 'Агент',
7: 'Брокер',
8: 'СТОА',
9: 'Контрагент',
10: 'Суброгация',
11: 'Ипотека',
12: 'Запросы СБ',
13: 'Запросы УРМ',
14: 'ДЗ',
15: 'Осмотрщик',
16: 'Дилер',
17: 'ЛПУ',
18: 'Тоталь',
19: 'ДМС',
20: 'Арбитражный управляющий',
21: 'НС',
22: 'Страхование ответственности',
23: 'Тендер'}

sources = {
1: 'DI',
2: 'Агентский',
3: 'Брокерский',
4: 'Дилерский',
5: 'Корпоративный',
6: 'Банковский',
7: 'Лизинг',
8: 'Интач',
9: 'ЦОК',
10: 'Агентский',
11: 'Брокерский',
12: 'Дилерский',
13: 'Корпоративный',
14: 'Банковский',
15: 'Лизинг',
16: 'Тендер',
17: 'Штатный кандидат',
18: 'Аутсорсинг',
19: 'Стажер',
20: 'Тинькофф',
21: 'Иное',
0: 'Иное'}

df['Check_type'] = df['CHECK_TYPE_ID'].map(check_types)

df['Source'] = df['SRC_ID'].map(sources)

# Класс BusinessHours

In [994]:
import datetime


class BusinessHours:

    def __init__(self, datetime1, datetime2, worktiming=[8, 20]
                , weekends=[6, 7]):
        self.weekends = weekends
        self.worktiming = worktiming
        self.datetime1 = datetime1
        self.datetime2 = datetime2
        self.day_hours = (self.worktiming[1]-self.worktiming[0])
        self.day_minutes = self.day_hours * 60 # minutes in a work day

    def getdays(self):
        return int(self.getminutes() / self.day_minutes)
    
    def gethours(self):
        return int(self.getminutes() / 60)

    def getminutes(self):
        """
        Return the difference in minutes.
        """
        # 
        dt_start = self.datetime1  # datetime of start
        dt_end = self.datetime2    # datetime of end
        worktime_in_seconds = 0

        if dt_start.date() == dt_end.date():
            # starts and ends on same workday
            full_days = 0
            if self.is_weekend(dt_start):
                return 0
            else:
                if dt_start.hour < self.worktiming[0]:
                    # 
                    dt_start = datetime.datetime(
                        year=dt_start.year,
                        month=dt_start.month,
                        day=dt_start.day,
                        hour=self.worktiming[0],
                        minute=0)
                if dt_start.hour >= self.worktiming[1] or \
                        dt_end.hour < self.worktiming[0]:
                    return 0
                if dt_end.hour >= self.worktiming[1]:
                    dt_end = datetime.datetime(
                        year=dt_end.year,
                        month=dt_end.month,
                        day=dt_end.day,
                        hour=self.worktiming[1],
                        minute=0)
                worktime_in_seconds = (dt_end-dt_start).total_seconds()
        elif (dt_end-dt_start).days < 0:
            # ends before start
            return 0
        else:
            # start and ends on different days
            current_day = dt_start  # 
            while not current_day.date() == dt_end.date():
                if not self.is_weekend(current_day):
                    if current_day == dt_start:
                        # 
                        if current_day.hour < self.worktiming[0]:
                            # starts before the work day
                            worktime_in_seconds += self.day_minutes*60  # add 1 full work day
                        elif current_day.hour >= self.worktiming[1]:
                            pass  # 
                        else:
                            # 
                            dt_currentday_close = datetime.datetime(
                                year=dt_start.year,
                                month=dt_start.month,
                                day=dt_start.day,
                                hour=self.worktiming[1],
                                minute=0)
                            worktime_in_seconds += (dt_currentday_close
                                         - dt_start).total_seconds()
                    else:
                        # 
                        worktime_in_seconds += self.day_minutes*60
                current_day += datetime.timedelta(days=1)  # next day
            # Time on the last day
            if not self.is_weekend(dt_end):
                if dt_end.hour >= self.worktiming[1]:  # finish after close
                    # Add a full day
                    worktime_in_seconds += self.day_minutes*60
                elif dt_end.hour < self.worktiming[0]:  # close before opening
                    pass  # no time added
                else:
                    #
                    dt_end_open = datetime.datetime(
                        year=dt_end.year,
                        month=dt_end.month,
                        day=dt_end.day,
                        hour=self.worktiming[0],
                        minute=0)
                    worktime_in_seconds += (dt_end-dt_end_open).total_seconds()
        return int(worktime_in_seconds/60)

    def is_weekend(self, datetime):
        """
        Returns True if datetime is a weekend.
        """
        #for weekend in self.weekends:
        if datetime.date() in auto_weekends['DATE'].dt.date.values:
            #if datetime.isoweekday() == weekend:
            return True
        return False

# Класс BusinessHours для дургих типов проверок

In [995]:
import datetime

class RestBusinessHours:

    def __init__(self, datetime1, datetime2, worktiming=[8, 20]
                , weekends=[6, 7]):
        self.weekends = weekends
        self.worktiming = worktiming
        self.datetime1 = datetime1
        self.datetime2 = datetime2
        self.day_hours = (self.worktiming[1]-self.worktiming[0])
        self.day_minutes = self.day_hours * 60 # minutes in a work day

    def getdays(self):
        return int(self.getminutes() / self.day_minutes)
    
    def gethours(self):
        return int(self.getminutes() / 60)

    def getminutes(self):
        """
        Return the difference in minutes.
        """
        # 
        dt_start = self.datetime1  # datetime of start
        dt_end = self.datetime2    # datetime of end
        worktime_in_seconds = 0

        if dt_start.date() == dt_end.date():
            # starts and ends on same workday
            full_days = 0
            if self.is_weekend(dt_start):
                return 0
            else:
                if dt_start.hour < self.worktiming[0]:
                    # set start time to opening hour
                    dt_start = datetime.datetime(
                        year=dt_start.year,
                        month=dt_start.month,
                        day=dt_start.day,
                        hour=self.worktiming[0],
                        minute=0)
                if dt_start.hour >= self.worktiming[1] or \
                        dt_end.hour < self.worktiming[0]:
                    return 0
                if dt_end.hour >= self.worktiming[1]:
                    dt_end = datetime.datetime(
                        year=dt_end.year,
                        month=dt_end.month,
                        day=dt_end.day,
                        hour=self.worktiming[1],
                        minute=0)
                worktime_in_seconds = (dt_end-dt_start).total_seconds()
        elif (dt_end-dt_start).days < 0:
            # ends before start
            return 0
        else:
            # start and ends on different days
            current_day = dt_start  # 
            while not current_day.date() == dt_end.date():
                if not self.is_weekend(current_day):
                    if current_day == dt_start:
                        # 
                        if current_day.hour < self.worktiming[0]:
                            # starts before the work day
                            worktime_in_seconds += self.day_minutes*60  # 
                        elif current_day.hour >= self.worktiming[1]:
                            pass  # 
                        else:
                            # 
                            dt_currentday_close = datetime.datetime(
                                year=dt_start.year,
                                month=dt_start.month,
                                day=dt_start.day,
                                hour=self.worktiming[1],
                                minute=0)
                            worktime_in_seconds += (dt_currentday_close
                                         - dt_start).total_seconds()
                    else:
                        # 
                        worktime_in_seconds += self.day_minutes*60
                current_day += datetime.timedelta(days=1)  # next day
            # Time on the last day
            if not self.is_weekend(dt_end):
                if dt_end.hour >= self.worktiming[1]:  # finish after close
                    # Add a full day
                    worktime_in_seconds += self.day_minutes*60
                elif dt_end.hour < self.worktiming[0]:  # close before opening
                    pass  # no time added
                else:
                    # Add time since opening
                    dt_end_open = datetime.datetime(
                        year=dt_end.year,
                        month=dt_end.month,
                        day=dt_end.day,
                        hour=self.worktiming[0],
                        minute=0)
                    worktime_in_seconds += (dt_end-dt_end_open).total_seconds()
        return int(worktime_in_seconds/60)

    def is_weekend(self, datetime):
        """
        Returns True if datetime is a weekend.
        """
        #for weekend in self.weekends:
        if datetime.date() in rest_weekends['DATE'].dt.date.values:
            #if datetime.isoweekday() == weekend:
            return True
        return False

# Считаем время обработки заявки

In [996]:
%%time

diff = []
for i, j, k in zip(df['DT_LETTER'], df['DT_DECISION'], df['CHECK_TYPE_ID']):
    try:
        if k == 1:
            diff.append(BusinessHours(i, j).getminutes()/60)
        else:
            diff.append(RestBusinessHours(i, j).getminutes()/60)
    except :
        diff.append('NAN')
        
df['time_diff'] = diff

CPU times: user 1min 32s, sys: 139 ms, total: 1min 32s
Wall time: 1min 32s


In [997]:
df = df[df['time_diff'] != 'NAN']

df['time_diff'] = pd.to_numeric(df['time_diff'])

In [998]:
df

,ID,CATEGORY_ID,CHECK_TYPE_ID,SRC_ID,DT_LETTER,DT_DECISION,week,Check_type,Source,time_diff
1860,106359,ФЛ,1,0,2017-12-01 07:30:00,2017-12-01 08:29:15,27.11.2017-03.12.2017,АВТО,Иное,0.483333
3500,106371,ФЛ,1,0,2017-12-01 07:40:00,2017-12-01 09:30:08,27.11.2017-03.12.2017,АВТО,Иное,1.500000
2409,106357,ФЛ,1,1,2017-12-01 08:09:00,2017-12-01 08:18:17,27.11.2017-03.12.2017,АВТО,DI,0.150000
1845,106362,ФЛ,1,0,2017-12-01 08:29:00,2017-12-01 08:53:22,27.11.2017-03.12.2017,АВТО,Иное,0.400000
3499,106367,ФЛ,1,1,2017-12-01 08:58:00,2017-12-01 09:12:27,27.11.2017-03.12.2017,АВТО,DI,0.233333
2467,106376,ЮЛ,2,0,2017-12-01 09:14:00,2017-12-01 10:19:00,27.11.2017-03.12.2017,Перевозчик грузов,Иное,1.083333
3057,106397,ЮЛ,8,0,2017-12-01 09:23:00,2017-12-01 11:51:05,27.11.2017-03.12.2017,СТОА,Иное,2.466667
3501,106373,ФЛ,1,0,2017-12-01 09:44:00,2017-12-01 09:56:45,27.11.2017-03.12.2017,АВТО,Иное,0.200000
2468,106377,ФЛ,1,1,2017-12-01 09:47:00,2017-12-01 10:11:18,27.11.2017-03.12.2017,АВТО,DI,0.400000
3065,106410,ЮЛ,7,0,2017-12-01 09:49:00,2017-12-01 12:27:18,27.11.2017-03.12.2017,Брокер,Иное,2.633333


In [999]:
df.shape

(51432, 10)

In [1000]:
len(diff)

51437

In [1001]:
df.head()

,ID,CATEGORY_ID,CHECK_TYPE_ID,SRC_ID,DT_LETTER,DT_DECISION,week,Check_type,Source,time_diff
1860,106359,ФЛ,1,0,2017-12-01 07:30:00,2017-12-01 08:29:15,27.11.2017-03.12.2017,АВТО,Иное,0.483333
3500,106371,ФЛ,1,0,2017-12-01 07:40:00,2017-12-01 09:30:08,27.11.2017-03.12.2017,АВТО,Иное,1.500000
2409,106357,ФЛ,1,1,2017-12-01 08:09:00,2017-12-01 08:18:17,27.11.2017-03.12.2017,АВТО,DI,0.150000
1845,106362,ФЛ,1,0,2017-12-01 08:29:00,2017-12-01 08:53:22,27.11.2017-03.12.2017,АВТО,Иное,0.400000
3499,106367,ФЛ,1,1,2017-12-01 08:58:00,2017-12-01 09:12:27,27.11.2017-03.12.2017,АВТО,DI,0.233333


In [1002]:
def cut_date(x):
    return pd.to_datetime(x[:10], dayfirst= True)

def week_stop(x):
    return pd.to_datetime(x[11:], dayfirst= True)

In [1003]:
df[(df['Check_type'] == 'АВТО') & (df['time_diff'] <= 8)].shape

(40696, 10)

In [1004]:
AUTO.shape

(40717, 11)

# Все АВТО

In [1005]:
AUTO = df[(df['Check_type'] == 'АВТО') & (df['time_diff'] <= 8)]

#AUTO =  df[(df['Check_type'] == 'АВТО') &  (df['Source'] != 'DI')]

AUTO['is_long'] = np.where(AUTO['time_diff'] > 3, 1, 0) # больше ли регламентированного времени

AUTO_grouped = AUTO.groupby('week')

mean_median_AUTO = AUTO_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_AUTO = (AUTO_grouped['is_long'].sum() / AUTO_grouped['is_long'].count()).reset_index()

AUTO_final = pd.merge(mean_median_AUTO, max_portion_AUTO, how ='inner', on='week') # concat two tables

AUTO_final['week_start'] = AUTO_final['week'].apply(cut_date) # apply function cut_date to sort values

AUTO_final['week_stop'] = AUTO_final['week'].apply(week_stop)
 
AUTO_final = AUTO_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [1006]:
AUTO_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
47,27.11.2017-03.12.2017,2.049320,1.616667,0.0,7.500000,0.210884,2017-11-27,2017-12-03
6,04.12.2017-10.12.2017,1.504854,1.050000,0.0,7.883333,0.156385,2017-12-04,2017-12-10
18,11.12.2017-17.12.2017,1.182481,0.783333,0.0,4.366667,0.060565,2017-12-11,2017-12-17
30,18.12.2017-24.12.2017,1.525717,0.883333,0.0,7.833333,0.161212,2017-12-18,2017-12-24
42,25.12.2017-31.12.2017,1.250123,0.850000,0.0,6.883333,0.081361,2017-12-25,2017-12-31


# Авто агентские

In [1007]:
AGENT = df[(df['Check_type'] == 'АВТО') & (df['Source'] == 'Агентский')  & (df['CATEGORY_ID'] == 'ФЛ') 
           & (df['time_diff'] <= 8)]

AGENT['is_long'] = np.where(AGENT['time_diff'] > 1, 1, 0) # больше ли регламентированного времени

AGENT_grouped = AGENT.groupby('week')

mean_median_AGENT = AGENT_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_AGENT = (AGENT_grouped['is_long'].sum() / AGENT_grouped['is_long'].count()).reset_index()

AGENT_final = pd.merge(mean_median_AGENT, max_portion_AGENT, how ='inner', on='week') # concat two tables

AGENT_final['week_start'] = AGENT_final['week'].apply(cut_date) # apply function cut_date to sort values

AGENT_final['week_stop'] = AGENT_final['week'].apply(week_stop)
 
AGENT_final = AGENT_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [1008]:
AGENT_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
32,26.02.2018-04.03.2018,0.750971,0.533333,0.100000,5.616667,0.174757,2018-02-26,2018-03-04
5,05.03.2018-11.03.2018,1.143553,0.675000,0.016667,6.000000,0.273585,2018-03-05,2018-03-11
14,12.03.2018-18.03.2018,0.904292,0.633333,0.066667,5.133333,0.270386,2018-03-12,2018-03-18
23,19.03.2018-25.03.2018,1.087216,0.550000,0.083333,6.916667,0.269076,2018-03-19,2018-03-25
33,26.03.2018-01.04.2018,0.673579,0.483333,0.000000,5.683333,0.133641,2018-03-26,2018-04-01


# DI

In [1009]:
DI = df[(df['Check_type'] == 'АВТО') & (df['Source'] == 'DI') & (df['CATEGORY_ID'] == 'ФЛ') & (df['time_diff'] <= 8)]

DI['is_long'] = np.where(DI['time_diff'] > 1, 1, 0) # больше ли регламентированного времени

DI_grouped = DI.groupby('week')

mean_median_DI = DI_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_DI = (DI_grouped['is_long'].sum() / DI_grouped['is_long'].count()).reset_index()

DI_final = pd.merge(mean_median_DI, max_portion_DI, how ='inner', on='week') # concat two tables

DI_final['week_start'] = DI_final['week'].apply(cut_date) # apply function cut_date to sort values

DI_final['week_stop'] = DI_final['week'].apply(week_stop)
 
DI_final = DI_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1010]:
DI_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
47,27.11.2017-03.12.2017,1.669697,1.116667,0.150000,4.350000,0.568182,2017-11-27,2017-12-03
6,04.12.2017-10.12.2017,0.875733,0.700000,0.133333,7.883333,0.336000,2017-12-04,2017-12-10
18,11.12.2017-17.12.2017,0.565840,0.450000,0.000000,2.500000,0.099174,2017-12-11,2017-12-17
30,18.12.2017-24.12.2017,1.543669,0.866667,0.083333,5.766667,0.457364,2017-12-18,2017-12-24
42,25.12.2017-31.12.2017,1.022994,0.750000,0.000000,3.616667,0.398148,2017-12-25,2017-12-31


# Дилеры

In [1011]:
DEALER = df[(df['Check_type'] == 'АВТО') & (df['Source'] == 'Дилерский')  & (df['CATEGORY_ID'] == 'ФЛ') 
            & (df['time_diff'] <= 8)]

DEALER['is_long'] = np.where(DEALER['time_diff'] > 3, 1, 0) # больше ли регламентированного времени

DEALER_grouped = DEALER.groupby('week')

mean_median_DEALER = DEALER_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_DEALER = (DEALER_grouped['is_long'].sum() / DEALER_grouped['is_long'].count()).reset_index()

DEALER_final = pd.merge(mean_median_DEALER, max_portion_DEALER, how ='inner', on='week') # concat two tables

DEALER_final['week_start'] = DEALER_final['week'].apply(cut_date) # apply function cut_date to sort values

DEALER_final['week_stop'] = DEALER_final['week'].apply(week_stop)
 
DEALER_final = DEALER_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [1012]:
DEALER_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
32,26.02.2018-04.03.2018,2.078667,1.983333,0.233333,4.850000,0.260000,2018-02-26,2018-03-04
5,05.03.2018-11.03.2018,3.507917,2.875000,0.150000,7.216667,0.450000,2018-03-05,2018-03-11
14,12.03.2018-18.03.2018,2.227576,1.691667,0.250000,7.766667,0.218182,2018-03-12,2018-03-18
23,19.03.2018-25.03.2018,3.068692,2.716667,0.150000,8.000000,0.429907,2018-03-19,2018-03-25
33,26.03.2018-01.04.2018,1.282960,0.975000,0.083333,5.083333,0.044776,2018-03-26,2018-04-01


# БРОКЕРЫ

In [1013]:
BROKER = df[(df['Check_type'] == 'АВТО') & (df['Source'] == 'Брокерский')  & (df['CATEGORY_ID'] == 'ФЛ') 
            & (df['time_diff'] <= 8)]

BROKER['is_long'] = np.where(BROKER['time_diff'] > 3, 1, 0) # больше ли регламентированного времени

BROKER_grouped = BROKER.groupby('week')

mean_median_BROKER = BROKER_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_BROKER = (BROKER_grouped['is_long'].sum() / BROKER_grouped['is_long'].count()).reset_index()

BROKER_final = pd.merge(mean_median_BROKER, max_portion_BROKER, how ='inner', on='week') # concat two tables

BROKER_final['week_start'] = BROKER_final['week'].apply(cut_date) # apply function cut_date to sort values

BROKER_final['week_stop'] = BROKER_final['week'].apply(week_stop)
 
BROKER_final = BROKER_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [1014]:
BROKER_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
32,26.02.2018-04.03.2018,2.101136,1.783333,0.433333,4.200000,0.250000,2018-02-26,2018-03-04
5,05.03.2018-11.03.2018,3.832527,3.200000,0.333333,7.283333,0.564516,2018-03-05,2018-03-11
14,12.03.2018-18.03.2018,2.481897,2.025000,0.216667,8.000000,0.293103,2018-03-12,2018-03-18
23,19.03.2018-25.03.2018,3.184512,2.850000,0.166667,7.733333,0.424242,2018-03-19,2018-03-25
33,26.03.2018-01.04.2018,1.348382,1.133333,0.000000,4.166667,0.058252,2018-03-26,2018-04-01


# Парк ТС

In [1015]:
PARK_TS = df[(df['Check_type'] == 'Парк ТС')]

PARK_TS['is_long'] = np.where(PARK_TS['time_diff'] > 24, 1, 0) # больше ли регламентированного времени

PARK_TS_grouped = PARK_TS.groupby('week')

mean_median_PARK_TS = PARK_TS_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_PARK_TS = (PARK_TS_grouped['is_long'].sum() / PARK_TS_grouped['is_long'].count()).reset_index()

PARK_TS_final = pd.merge(mean_median_PARK_TS, max_portion_PARK_TS, how ='inner', on='week') # concat two tables

PARK_TS_final['week_start'] = PARK_TS_final['week'].apply(cut_date) # apply function cut_date to sort values

PARK_TS_final['week_stop'] = PARK_TS_final['week'].apply(week_stop)
 
PARK_TS_final = PARK_TS_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1016]:
PARK_TS_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
46,27.11.2017-03.12.2017,40.933333,40.933333,39.866667,42.000000,1.0,2017-11-27,2017-12-03
5,04.12.2017-10.12.2017,6.809524,1.416667,0.566667,22.933333,0.0,2017-12-04,2017-12-10
17,11.12.2017-17.12.2017,8.376190,7.850000,1.566667,20.383333,0.0,2017-12-11,2017-12-17
29,18.12.2017-24.12.2017,6.262963,6.250000,0.766667,10.250000,0.0,2017-12-18,2017-12-24
41,25.12.2017-31.12.2017,1.558333,1.558333,0.450000,2.666667,0.0,2017-12-25,2017-12-31


# Перевозчики грузов

In [1017]:
GRUZ = df[(df['Check_type'] == 'Перевозчик грузов') & (df['time_diff'] <= 8)]

GRUZ['is_long'] = np.where(GRUZ['time_diff'] > 4, 1, 0) # больше ли регламентированного времени

GRUZ_grouped = GRUZ.groupby('week')

mean_median_GRUZ = GRUZ_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_GRUZ = (GRUZ_grouped['is_long'].sum() / GRUZ_grouped['is_long'].count()).reset_index()

GRUZ_final = pd.merge(mean_median_GRUZ, max_portion_GRUZ, how ='inner', on='week') # concat two tables

GRUZ_final['week_start'] = GRUZ_final['week'].apply(cut_date) # apply function cut_date to sort values

GRUZ_final['week_stop'] = GRUZ_final['week'].apply(week_stop)
 
GRUZ_final = GRUZ_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1018]:
GRUZ_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
46,27.11.2017-03.12.2017,2.400000,1.116667,0.333333,7.833333,0.214286,2017-11-27,2017-12-03
5,04.12.2017-10.12.2017,1.301016,0.900000,0.200000,5.966667,0.060976,2017-12-04,2017-12-10
17,11.12.2017-17.12.2017,1.006777,0.566667,0.133333,7.700000,0.054945,2017-12-11,2017-12-17
29,18.12.2017-24.12.2017,1.674603,1.033333,0.133333,6.516667,0.104762,2017-12-18,2017-12-24
41,25.12.2017-31.12.2017,1.919309,1.541667,0.233333,7.800000,0.109756,2017-12-25,2017-12-31


# Имущество ФЛ

In [1019]:
IMUSHESTVO_FL = df[(df['Check_type'] == 'Имущество') & (df['CATEGORY_ID'] == 'ФЛ')]

IMUSHESTVO_FL['is_long'] = np.where(IMUSHESTVO_FL['time_diff'] > 24, 1, 0) # больше ли регламентированного времени

IMUSHESTVO_FL_grouped = IMUSHESTVO_FL.groupby('week')

mean_median_IMUSHESTVO_FL = IMUSHESTVO_FL_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_IMUSHESTVO_FL = (IMUSHESTVO_FL_grouped['is_long'].sum() / IMUSHESTVO_FL_grouped['is_long'].count()).reset_index()

IMUSHESTVO_FL_final = pd.merge(mean_median_IMUSHESTVO_FL, max_portion_IMUSHESTVO_FL, how ='inner', on='week') # concat two tables

IMUSHESTVO_FL_final['week_start'] = IMUSHESTVO_FL_final['week'].apply(cut_date) # apply function cut_date to sort values

IMUSHESTVO_FL_final['week_stop'] = IMUSHESTVO_FL_final['week'].apply(week_stop)
 
IMUSHESTVO_FL_final = IMUSHESTVO_FL_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1020]:
IMUSHESTVO_FL_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
5,04.12.2017-10.12.2017,13.804167,12.775000,11.283333,18.383333,0.0,2017-12-04,2017-12-10
16,11.12.2017-17.12.2017,8.955556,9.983333,1.533333,15.350000,0.0,2017-12-11,2017-12-17
27,18.12.2017-24.12.2017,34.116667,34.116667,32.316667,35.916667,1.0,2017-12-18,2017-12-24
38,25.12.2017-31.12.2017,9.219048,9.916667,0.433333,15.600000,0.0,2017-12-25,2017-12-31
10,08.01.2018-14.01.2018,6.816667,5.066667,1.516667,15.833333,0.0,2018-01-08,2018-01-14


# ИПОТЕКА

In [1021]:
IPOTEKA = df[(df['Check_type'] == 'Ипотека')]

IPOTEKA['is_long'] = np.where(IPOTEKA['time_diff'] > 8, 1, 0) # больше ли регламентированного времени

IPOTEKA_grouped = IPOTEKA.groupby('week')

mean_median_IPOTEKA = IPOTEKA_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_IPOTEKA = (IPOTEKA_grouped['is_long'].sum() / IPOTEKA_grouped['is_long'].count()).reset_index()

IPOTEKA_final = pd.merge(mean_median_IPOTEKA, max_portion_IPOTEKA, how ='inner', on='week') # concat two tables

IPOTEKA_final['week_start'] = IPOTEKA_final['week'].apply(cut_date) # apply function cut_date to sort values

IPOTEKA_final['week_stop'] = IPOTEKA_final['week'].apply(week_stop)
 
IPOTEKA_final = IPOTEKA_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1022]:
IPOTEKA_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
5,04.12.2017-10.12.2017,8.750000,8.750000,8.750000,8.750000,1.0,2017-12-04,2017-12-10
13,11.12.2017-17.12.2017,1.308333,1.133333,0.983333,1.983333,0.0,2017-12-11,2017-12-17
25,18.12.2017-24.12.2017,11.483333,11.483333,11.483333,11.483333,1.0,2017-12-18,2017-12-24
36,25.12.2017-31.12.2017,1.383333,1.383333,1.383333,1.383333,0.0,2017-12-25,2017-12-31
9,08.01.2018-14.01.2018,2.205556,1.833333,0.366667,4.416667,0.0,2018-01-08,2018-01-14


# СТРАХОВАТЕЛИ ЮЛ

In [1023]:
imushestvo_not_fl = df[(df['Check_type'] == 'Имущество') & (df['CATEGORY_ID'] != 'ФЛ')]

rest = df[(df['Check_type'] == 'Страхование ответственности') | (df['Check_type'] == 'Арбитражный управляющий')]

STRAHOVATELI = pd.concat([imushestvo_not_fl, rest])

STRAHOVATELI['is_long'] = np.where(STRAHOVATELI['time_diff'] > 24, 1, 0) # больше ли регламентированного времени

STRAHOVATELI_grouped = STRAHOVATELI.groupby('week')

mean_median_STRAHOVATELI = STRAHOVATELI_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_STRAHOVATELI = (STRAHOVATELI_grouped['is_long'].sum() / STRAHOVATELI_grouped['is_long'].count()).reset_index()

STRAHOVATELI_final = pd.merge(mean_median_STRAHOVATELI, max_portion_STRAHOVATELI, how ='inner', on='week') # concat two tables

STRAHOVATELI_final['week_start'] = STRAHOVATELI_final['week'].apply(cut_date) # apply function cut_date to sort values

STRAHOVATELI_final['week_stop'] = STRAHOVATELI_final['week'].apply(week_stop)
 
STRAHOVATELI_final = STRAHOVATELI_final.sort_values(by='week_start')

In [1024]:
STRAHOVATELI_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
45,27.11.2017-03.12.2017,3.650000,3.650000,3.650000,3.650000,0.0,2017-11-27,2017-12-03
5,04.12.2017-10.12.2017,7.466667,8.150000,3.150000,11.100000,0.0,2017-12-04,2017-12-10
17,11.12.2017-17.12.2017,7.629412,4.466667,0.550000,17.950000,0.0,2017-12-11,2017-12-17
28,18.12.2017-24.12.2017,7.123333,1.916667,0.466667,17.083333,0.0,2017-12-18,2017-12-24
40,25.12.2017-31.12.2017,7.150000,6.766667,0.483333,15.666667,0.0,2017-12-25,2017-12-31


# Кандидат

In [1025]:
candidate = df[(df['Check_type'] == 'Кандидат') & (df['time_diff'] <= 48)]

osmotrshik = df[(df['Check_type'] == 'Осмотрщик') & (df['CATEGORY_ID'] == 'ФЛ') & (df['time_diff'] <= 48)]

CANDIDATE = pd.concat([candidate, osmotrshik])

CANDIDATE['is_long'] = np.where(CANDIDATE['time_diff'] > 24, 1, 0) # больше ли регламентированного времени

CANDIDATE_grouped = CANDIDATE.groupby('week')

mean_median_CANDIDATE = CANDIDATE_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_CANDIDATE = (CANDIDATE_grouped['is_long'].sum() / CANDIDATE_grouped['is_long'].count()).reset_index()

CANDIDATE_final = pd.merge(mean_median_CANDIDATE, max_portion_CANDIDATE, how ='inner', on='week') # concat two tables

CANDIDATE_final['week_start'] = CANDIDATE_final['week'].apply(cut_date) # apply function cut_date to sort values

CANDIDATE_final['week_stop'] = CANDIDATE_final['week'].apply(week_stop)
 
CANDIDATE_final = CANDIDATE_final.sort_values(by='week_start')

In [1026]:
CANDIDATE_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
46,27.11.2017-03.12.2017,15.628571,17.033333,2.016667,19.666667,0.000000,2017-11-27,2017-12-03
5,04.12.2017-10.12.2017,14.013596,12.725000,0.716667,35.000000,0.236842,2017-12-04,2017-12-10
17,11.12.2017-17.12.2017,8.793537,7.150000,0.400000,28.583333,0.102041,2017-12-11,2017-12-17
29,18.12.2017-24.12.2017,7.157051,1.575000,0.550000,33.783333,0.076923,2017-12-18,2017-12-24
41,25.12.2017-31.12.2017,6.529167,4.883333,0.550000,17.983333,0.000000,2017-12-25,2017-12-31


# Агент ФЛ и ИП

In [1027]:
AGENT_FL = df[(df['Check_type'] == 'Агент') & (df['CATEGORY_ID'] != 'ЮЛ') & (df['time_diff'] <= 48)]

AGENT_FL['is_long'] = np.where(AGENT_FL['time_diff'] > 24, 1, 0) # больше ли регламентированного времени

AGENT_FL_grouped = AGENT_FL.groupby('week')

mean_median_AGENT_FL = AGENT_FL_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_AGENT_FL = (AGENT_FL_grouped['is_long'].sum() / AGENT_FL_grouped['is_long'].count()).reset_index()

AGENT_FL_final = pd.merge(mean_median_AGENT_FL, max_portion_AGENT_FL, how ='inner', on='week') # concat two tables

AGENT_FL_final['week_start'] = AGENT_FL_final['week'].apply(cut_date) # apply function cut_date to sort values

AGENT_FL_final['week_stop'] = AGENT_FL_final['week'].apply(week_stop)
 
AGENT_FL_final = AGENT_FL_final.sort_values(by='week_start')

/opt/ds/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [1028]:
AGENT_FL_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
43,27.11.2017-03.12.2017,35.730556,35.641667,34.966667,36.583333,1.000000,2017-11-27,2017-12-03
4,04.12.2017-10.12.2017,16.526282,14.583333,4.050000,34.883333,0.153846,2017-12-04,2017-12-10
15,11.12.2017-17.12.2017,15.888889,19.575000,0.300000,30.333333,0.100000,2017-12-11,2017-12-17
27,18.12.2017-24.12.2017,12.016667,10.533333,0.333333,31.833333,0.157895,2017-12-18,2017-12-24
38,25.12.2017-31.12.2017,9.797222,9.575000,0.733333,21.433333,0.000000,2017-12-25,2017-12-31


# Посредники

In [1029]:
part_1 = df[(df['Check_type'] == 'Контрагент') | (df['Check_type'] == 'Брокер')| (df['Check_type'] == 'СТОА') |
  (df['Check_type'] == 'Дилер') | (df['Check_type'] == 'ДМС') | (df['Check_type'] == 'ЛПУ')
  |(df['Check_type'] == 'Тендер') | (df['Check_type'] == 'Тоталь')]

part_2 = df[(df['Check_type'] == 'Осмотрщик') & (df['CATEGORY_ID'] != 'ФЛ')]

part_3 = df[(df['Check_type'] == 'Агент') & (df['CATEGORY_ID'] == 'ЮЛ')] 

POSREDNIKI = pd.concat([part_1, part_2, part_3])

POSREDNIKI['is_long'] = np.where(POSREDNIKI['time_diff'] > 40, 1, 0) # больше ли регламентированного времени

POSREDNIKI_grouped = POSREDNIKI.groupby('week')

mean_median_POSREDNIKI = POSREDNIKI_grouped['time_diff'].agg(['mean', 'median', 'min', 'max']).reset_index()

max_portion_POSREDNIKI = (POSREDNIKI_grouped['is_long'].sum() / POSREDNIKI_grouped['is_long'].count()).reset_index()

POSREDNIKI_final = pd.merge(mean_median_POSREDNIKI, max_portion_POSREDNIKI, how ='inner', on='week') # concat two tables

POSREDNIKI_final['week_start'] = POSREDNIKI_final['week'].apply(cut_date) # apply function cut_date to sort values

POSREDNIKI_final['week_stop'] = POSREDNIKI_final['week'].apply(week_stop)
 
POSREDNIKI_final = POSREDNIKI_final.sort_values(by='week_start')

In [1030]:
POSREDNIKI_final.head()

,week,mean,median,min,max,is_long,week_start,week_stop
46,27.11.2017-03.12.2017,10.023333,9.608333,0.550000,19.600000,0.000000,2017-11-27,2017-12-03
5,04.12.2017-10.12.2017,18.703333,17.858333,3.350000,37.750000,0.000000,2017-12-04,2017-12-10
17,11.12.2017-17.12.2017,14.503571,17.083333,1.350000,22.950000,0.000000,2017-12-11,2017-12-17
29,18.12.2017-24.12.2017,21.080128,12.708333,1.950000,42.483333,0.115385,2017-12-18,2017-12-24
41,25.12.2017-31.12.2017,18.347368,19.650000,1.416667,37.683333,0.000000,2017-12-25,2017-12-31


In [1031]:
%%time
writer = pd.ExcelWriter("4 otchet KKK.xlsx")

AUTO_final.to_excel(writer,'АВТО ФЛ', index=False)

AGENT_final.to_excel(writer,'Авто Агентские', index=False)

DI_final.to_excel(writer,'Авто DI', index=False)

DEALER_final.to_excel(writer,'Авто Дилерский', index=False)

BROKER_final.to_excel(writer,'Авто Брокеры', index=False)

PARK_TS_final.to_excel(writer,'Парк ТС', index=False)

GRUZ_final.to_excel(writer,'Перевозчики грузов', index=False)

IMUSHESTVO_FL_final.to_excel(writer,'Имущество ФЛ', index=False)

IPOTEKA_final.to_excel(writer,'Ипотека', index=False)

STRAHOVATELI_final.to_excel(writer,'Страхователи ЮЛ', index=False)

CANDIDATE_final.to_excel(writer,'Кандидат', index=False)

AGENT_FL_final.to_excel(writer,'Агенты ФЛ и ИП', index=False)

POSREDNIKI_final.to_excel(writer,'Посредники', index=False)

df.to_excel(writer,'Все записи', index=False)

writer.save()

CPU times: user 15.3 s, sys: 299 ms, total: 15.6 s
Wall time: 15.6 s


In [1032]:
#df.shape

In [1033]:
#df[(df['Check_type'] == 'АВТО') & (df['time_diff'] > 8)]

In [1034]:
A = [1, 1]
B = [3, 2]

In [1035]:
for m,n in (A, B):
    print(m)

1
3


In [1036]:
#import cx_Oracle

In [1037]:

URM = 
  (DESCRIPTION = 
    (ADDRESS = 
      (PROTOCOL = TCP)
      (HOST = renmskorurm02.mos.renins.com)
      (PORT = 1521)
    )
    (CONNECT_DATA = 
      (SERVICE_NAME = URM)
    )
  )


SyntaxError: invalid syntax (<ipython-input-1037-ef37333c528b>, line 2)

In [ ]:
"""host = 'renmskorurm02.mos.renins.com'
port = 1521
service_name = 'URM'

dsn_tns = cx_Oracle.makedsn(host, port, service_name)

db = cx_Oracle.connect('balbasa', 'Rjhbfylh', dsn_tns)"""

In [ ]:
"""
cx_Oracle.connect(user='muk12345', password='12345678q', dsn=None, mode=None, handle=None, pool=None, threaded=False, 
                  events=False, cclass=None, purity=None, newpassword=None, encoding=None, nencoding=None, edition=None,
                  appcontext=[], tag=None, matchanytag=None, shardingkey=[], supershardingkey=[])
"""

In [ ]:
#print (dsn_tns)

In [ ]:
#!pip install wheel

In [ ]:
#!pip install pyinstaller

In [ ]:
#!pyinstaller --onedir --onefile --name=downblog "/home/ds/zholtma/KKK отчет/Отчет _Время ответа_.py"